# Apache Spark intro


Apache Spark is a general-purpose, in-memory computing engine. Spark can be used with Hadoop, Yarn and other Big Data components to harness the power of Spark and improve the performance of your applications. It provides high-level APIs in Scala, Java, Python, R, and SQL.

**Spark Architecture**

Apache Spark works in a master-slave architecture where the master is called “Driver” and slaves are called “Workers”. The starting point of your Spark application is `sc`, a Spark Context Class instance. It runs inside the driver.

![](https://miro.medium.com/v2/resize:fit:1192/0*XzNeTtwEgIy5yWR_)


![](https://miro.medium.com/v2/resize:fit:1266/0*-PltnPR9row8iUDo)

**Apache Spark**: 
Sometimes also called Spark Core. The Spark Core implementation is a RDD (Resilient Distributed Dataset) which is a collection of distributed data across different nodes of the cluster that are processed in parallel.


**Spark SQL**: 
The implementation here is DataFrame, which is a relational representation of the data. It provides functions with SQL like capabilities. Also, we can write SQL like queries for our data analysis.


**Spark Streaming**: 
The implementation provided by this library is D-stream, also called Discretized Stream. This library provides capabilities to process/transform data in near real-time.


**MLlib**: 
This is a Machine Learning library with commonly used algorithms including collaborative filtering, classification, clustering, and regression.

**GraphX**: 
This library helps us to process Graphs, solving various problems (like Page Rank, Connected Components, etc) using Graph Theory.


Let’s dig a little deeper into Apache Spark (Spark Core), starting with RDD.




In [ ]:
from pyspark import SparkContext

In [ ]:
sc = SparkContext(appName="myAppName")

In [ ]:
sc

### RDD

- Resilient Distributed Dataset
- Podstawowa abstrakcja oraz rdzeń Sparka
- Obsługiwane przez dwa rodzaje operacji:
    - Akcje:
        - operacje uruchamiająceegzekucję transformacji na RDD
        - przyjmują RDD jako input i zwracają wynik NIE będący RDD
    - Transformacje:
        - leniwe operacje
        - przyjmują RDD i zwracają RDD

- In-Memory - dane RDD przechowywane w pamięci
- Immutable 
- Lazy evaluated
- Parallel - przetwarzane równolegle
- Partitioned - rozproszone 

## WAŻNE informacje !

Ważne do zrozumienia działania SPARKA:

Term                   |Definition
----                   |-------
RDD                    |Resilient Distributed Dataset
Transformation         |Spark operation that produces an RDD
Action                 |Spark operation that produces a local object
Spark Job              |Sequence of transformations on data with a final action


Dwie podstawowe metody tworzenia RDD:

Method                      |Result
----------                               |-------
`sc.parallelize(array)`                  |Create RDD of elements of array (or list)
`sc.textFile(path/to/file)`                      |Create RDD of lines from file

Podstawowe transformacje

Transformation Example                          |Result
----------                               |-------
`filter(lambda x: x % 2 == 0)`           |Discard non-even elements
`map(lambda x: x * 2)`                   |Multiply each RDD element by `2`
`map(lambda x: x.split())`               |Split each string into words
`flatMap(lambda x: x.split())`           |Split each string into words and flatten sequence
`sample(withReplacement=True,0.25)`      |Create sample of 25% of elements with replacement
`union(rdd)`                             |Append `rdd` to existing RDD
`distinct()`                             |Remove duplicates in RDD
`sortBy(lambda x: x, ascending=False)`   |Sort elements in descending order

Podstawowe akcje 

Action                             |Result
----------                             |-------
`collect()`                            |Convert RDD to in-memory list 
`take(3)`                              |First 3 elements of RDD 
`top(3)`                               |Top 3 elements of RDD
`takeSample(withReplacement=True,3)`   |Create sample of 3 elements with replacement
`sum()`                                |Find element sum (assumes numeric elements)
`mean()`                               |Find element mean (assumes numeric elements)
`stdev()`                              |Find element deviation (assumes numeric elements)

In [ ]:
keywords = ['Books', 'DVD', 'CD', 'PenDrive']
key_rdd = sc.parallelize(keywords)
key_rdd.collect()

In [ ]:
key_small = key_rdd.map(lambda x: x.lower()) # transformacja

In [ ]:
key_small.collect() # akcja 

In [ ]:
sc.stop()

**Spark’s core data structure**

✅: A low level object that lets Spark work its magic by splitting data across multiple nodes in the cluster.

❌: However, RDDs are hard to work with directly, so we’ll be using the Spark DataFrame abstraction built on top of RDDs.

## MAP REDUCE

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("new").getOrCreate()

# otrzymanie obiektu SparkContext
sc = spark.sparkContext

In [ ]:
import re
# Word Count on RDD 
sc.textFile("MobyDick.txt") \
.map(lambda x: re.findall(r"[a-z']+", x.lower())) \
.flatMap(lambda x: [(y, 1) for y in x]) \
.reduceByKey(lambda x,y: x + y) \
.collect()

In [ ]:
spark.stop()

## SPARK STREAMING

Część Sparka odpowiedzialna za przetwarzanie danych w czasie rzeczywistym. 


<img src="https://spark.apache.org/docs/latest/img/streaming-arch.png"/>

Dane mogą pochodzić z różnych źródeł np. sokety TCP, Kafka, etc. 
Korzystając z poznanych już metod `map, reduce, join, oraz window` można w łatwy sposób generować przetwarzanie strumienia tak jaby był to nieskończony ciąg RDD. 
Ponadto nie ma problemu aby wywołać na strumieniu operacje ML czy wykresy. 

Cała procedura przedstawia się następująco: 

<img src="https://spark.apache.org/docs/latest/img/streaming-flow.png"/>

SPARK STREAMING w tej wersji wprowadza abstrakcje zwaną `discretized stream` *DStream* (reprezentuje sekwencję RDD).

Operacje na DStream można wykonywać w API JAVA, SCALA, Python, R (nie wszystkie możliwości są dostępne dla Pythona). 


Spark Streaming potrzebuje minium 2 rdzenie.

----
- **StreamingContext(sparkContext, batchDuration)** - reprezentuje połączenie z klastrem i służy do tworzenia DStreamów, `batchDuration` wskazuje na granularność batch'y (w sekundach)
- **socketTextStream(hostname, port)** - tworzy DStream na podstawie danych napływających ze wskazanego źródła TCP
- **flatMap(f), map(f), reduceByKey(f)** - działają analogicznie jak w przypadku RDD z tym że tworzą nowe DStream'y
- **pprint(n)** - printuje pierwsze `n` (domyślnie 10) elementów z każdego RDD wygenerowanego w DStream'ie
- **StreamingContext.start()** - rozpoczyna działania na strumieniach
- **StreamingContext.awaitTermination(timeout)** - oczekuje na zakończenie działań na strumieniach
- **StreamingContext.stop(stopSparkContext, stopGraceFully)** - kończy działania na strumieniach

Obiekt StreamingContext można wygenerować za pomocą obiektu SparkContext.


<img src="https://spark.apache.org/docs/latest/img/streaming-dstream.png"/>

<img src="https://spark.apache.org/docs/latest/img/streaming-dstream-ops.png"/>

In [ ]:
import re
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

# Create a local StreamingContext with two working thread
# and batch interval of 1 second

sc = SparkContext("local[2]", "NetworkWordCount2")
ssc = StreamingContext(sc, 2)

# DStream
lines = ssc.socketTextStream("localhost", 9998)

# podziel każdą linię na wyrazy
# DStream jest mapowany na kolejny DStream
# words = lines.flatMap(lambda line: line.split(" "))

words = lines.flatMap(lambda x: re.findall(r"[a-z']+", x.lower()))

# zliczmy każdy wyraz w każdym batchu
# DStream jest mapowany na kolejny DStream
# pairs = words.map(lambda word: (word, 1))

# DStream jest mapowany na kolejny DStream                  
# wordCounts = pairs.reduceByKey(lambda x, y: x + y)

wordCounts = words.map(lambda word: (word,1)).reduceByKey(lambda x,y: x+y)
# wydrukuj pierwszy element
wordCounts.pprint()

In [ ]:
# before start run a stream data
ssc.start()             # Start the computation
ssc.awaitTermination()
ssc.stop()
sc.stop()

In [ ]:
# w konsoli linuxowej netcat Nmap for windows
!nc -lk 9998

In [ ]:
%%file start_stream.py

from socket import *
import time

rdd = list()
with open("MobyDick_full.txt", 'r') as ad:
    for line in ad:
        rdd.append(line)

HOST = 'localhost'
PORT = 9998
ADDR = (HOST, PORT)
tcpSock = socket(AF_INET, SOCK_STREAM)
tcpSock.bind(ADDR)
tcpSock.listen(5)


while True:
    c, addr = tcpSock.accept()
    print('got connection')
    for line in rdd:
        try:
            c.send(line.encode())
            time.sleep(1)
        except:
            break
    c.close()
    print('disconnected')


## nowe podejscie

Designed to behave a lot like a SQL table

✅:
- easier to understand,
- Operations using DataFrames are automatically optimized
- When using RDDs, it’s up to the data scientist to figure out the right way to optimize the query, but the DataFrame implementation has much of this optimization built in!

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
spark

### Przygotowanie danych

```bash
mkdir data
cd data
curl -L -o donation.zip http://bit.ly/1Aoywaq
unzip donation.zip
unzip 'block_*.zip'

```

In [ ]:
# create dataframe 
prev = spark.read.csv("data/block*.csv")

In [ ]:
prev

In [ ]:
prev.show(2)

In [ ]:
prev.show()

In [ ]:
# dodatkowe opcje z header i wartości null 
parsed = spark.read.option("header", "true")\
.option("nullValue", "?")\
.option("inferSchema", "true")\
.csv("data/block*.csv")

In [ ]:
parsed.show(5)

In [ ]:
parsed.printSchema()

## inne formaty 

- parquet
- orc
- json
- jdbc
- avro
- yrxy
- image
- libsvm
- binary
- xml

In [ ]:
parsed.write.format("parquet").save("data/block2.parquet")

In [ ]:
t = spark.read.format("parquet").load("data/block.parquet")

In [ ]:
t.show(2)

In [ ]:
spark.stop()

## schematy danych 

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType

schema = StructType([
  StructField("Date", StringType(), True),
  StructField("Open", DoubleType(), True),
  StructField("High", DoubleType(), True),
  StructField("Low", DoubleType(), True),
  StructField("Close", DoubleType(), True),
  StructField("Volume", IntegerType(), True),
  StructField("Name", StringType(), True)
])


ddlSchemaStr = """Date STRING, Open FLOAT, High FLOAT, 
Low FLOAT, Close FLOAT, Voulme INT, Name String 
"""

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

df = spark.read.option("header", True)\
.csv("data/stocks/AAPL_2006-01-01_to_2018-01-01.csv", schema=ddlSchemaStr)

df.show(5)

## dane niustrukturyzowane

In [ ]:
%%file test.json

{
 "id": "0001",
 "type": "donut",
 "name": "Cake",
 "ppu": 0.55,
 "batters":
  {
   "batter":
    [
     { "id": "1001", "type": "Regular" },
     { "id": "1002", "type": "Chocolate" },
     { "id": "1003", "type": "Blueberry" }
    ]
  },
 "topping":
  [
   { "id": "5001", "type": "None" },
   { "id": "5002", "type": "Glazed" },
   { "id": "5005", "type": "Sugar" },
   { "id": "5007", "type": "Powdered Sugar" },
   { "id": "5006", "type": "Chocolate with Sprinkles" },
   { "id": "5003", "type": "Chocolate" },
   { "id": "5004", "type": "Maple" }
  ]
}

In [ ]:
rawDFjson = spark.read.json("test.json", multiLine = "true")

In [ ]:
rawDFjson.printSchema()

In [ ]:
sampleDF = rawDFjson.withColumnRenamed("id", "key")

In [ ]:
sampleDF.printSchema()

In [ ]:
batDF = sampleDF.select("key", "batters.batter")
batDF.printSchema()

In [ ]:
batDF.show(1, False)

In [ ]:
from pyspark.sql.functions import explode
bat2DF = batDF.select("key", explode("batter").alias("new_batter"))
bat2DF.show()

In [ ]:
bat2DF.printSchema()

In [ ]:
bat2DF.select("key", "new_batter.*").show()

In [ ]:
finalBatDF = (sampleDF
        .select("key",  
explode("batters.batter").alias("new_batter"))
        .select("key", "new_batter.*")
        .withColumnRenamed("id", "bat_id")
        .withColumnRenamed("type", "bat_type"))
finalBatDF.show()

In [ ]:
topDF = (sampleDF
        .select("key", explode("topping").alias("new_topping"))
        .select("key", "new_topping.*")
        .withColumnRenamed("id", "top_id")
        .withColumnRenamed("type", "top_type")
        )
topDF.show(10, False)

## Eksploracyjna Analiza Danych

In [ ]:
# zweryfikuj schemat danych
parsed.printSchema()

In [ ]:
# sprawdz wartosci dla pierwszego rzedu
parsed.first()

In [ ]:
# ile przypadkow 
parsed.count()

In [ ]:
# zapisz do pamieci na klastrze (1 maszyna) 
parsed.cache()

In [ ]:
# target "is_match" liczba zgodnych i niezgodnych rekordow
from pyspark.sql.functions import col

parsed.groupBy("is_match").count().orderBy(col("count").desc()).show()

In [ ]:
# inne agregaty agg
from pyspark.sql.functions import avg, stddev, stddev_pop

parsed.agg(avg("cmp_sex"), stddev("cmp_sex"), stddev_pop("cmp_sex")).show()

In [ ]:
# polecenia sql - przypisanie nazwy dla silnika sql - tabela przejsciowa
parsed.createOrReplaceTempView("dane")

In [ ]:
spark.sql(""" SELECT is_match, COUNT(*) cnt FROM dane group by is_match order by cnt DESC""").show()

In [ ]:
# zbiorcze statystyki 
summary = parsed.describe()
summary.show()

In [ ]:
summary.select("summary", "cmp_fname_c1", "cmp_fname_c2").show()

> która zmienna lepiej opisze dane c1 czy c2 

In [ ]:
# statystyki dla poszczegolnych klas

# filtrowanie sql
matches = parsed.where("is_match = true")
# filtrowanie pyspark
misses = parsed.filter(col("is_match") == False)

match_summary = matches.describe()
miss_summary = misses.describe()

In [ ]:
match_summary.show()

In [ ]:
miss_summary.show()

## Tabele przestawne spark

In [ ]:
summary_p = summary.toPandas()

In [ ]:
summary_p.head()

In [ ]:
summary_p.shape

In [ ]:
summary_p = summary_p.set_index('summary').transpose().reset_index()
summary_p = summary_p.rename(columns={'index':'field'})
summary_p = summary_p.rename_axis(None, axis=1)

In [ ]:
summaryT = spark.createDataFrame(summary_p)
summaryT.show()

In [ ]:
summaryT.printSchema() # czy dobre typy danych ?? 

In [ ]:
from pyspark.sql.types import DoubleType

for c in summaryT.columns:
    if c == 'field':
        continue
    summaryT = summaryT.withColumn(c, summaryT[c].cast(DoubleType()))

In [ ]:
summaryT.printSchema() # teraz lepiej

In [ ]:
# wykonaj to samo dla tabel match i miss

In [ ]:
def pivot_summary(desc):
    desc_p = desc.toPandas()
    desc_p = desc_p.set_index('summary').transpose().reset_index()
    desc_p = desc_p.rename(columns={'index':'field'})
    desc_p = desc_p.rename_axis(None, axis=1)
    descT = spark.createDataFrame(desc_p)
    for c in descT.columns:
        if c == 'field':
            continue
        else:
            descT = descT.withColumn(c, descT[c].cast(DoubleType()))
    return descT

In [ ]:
match_summaryT = pivot_summary(match_summary)
miss_summaryT = pivot_summary(miss_summary)

## złączenia 

In [ ]:
match_summaryT.createOrReplaceTempView("match_s")
miss_summaryT.createOrReplaceTempView("miss_s")

In [ ]:
spark.sql("""
Select a.field, a.count + b.count total, a.mean - b.mean delta
from match_s a inner join miss_s b on a.field = b.field 
where a.field not in ("id_1", "id_2")
order by delta DESC, total DESC
""").show()

> do modelu : `cmp_plz`, `cmp_by`, `cmp_bd`, `cmp_lname_c1`, `cmp_bm`  

In [ ]:
## score = suma zmiennych
zmienne = ['cmp_plz','cmp_by','cmp_bd','cmp_lname_c1','cmp_bm']
suma = " + ".join(zmienne)

In [ ]:
suma

In [ ]:
from pyspark.sql.functions import expr

In [ ]:
scored = parsed.fillna(0, subset=zmienne)\
.withColumn('score', expr(suma))\
.select('score','is_match')

In [ ]:
scored.show()

In [ ]:
# ocena wartosci progowej
def crossTabs(scored, t):
    return scored.selectExpr(f"score >= {t} as above", "is_match")\
    .groupBy("above").pivot("is_match",("true","false"))\
    .count()

In [ ]:
crossTabs(scored, 4.0).show()

In [ ]:
crossTabs(scored, 2.0).show()